# Initial Prepration
1. Import of Data
2. Appending Test (Using mean as output class) and Training data

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from sklearn import model_selection, preprocessing

df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_extra = pd.read_csv('macro.csv')

# End Submission output 
output = pd.DataFrame()
output['id'] = df_test['id']

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
print(np.size(df_train, 0))
print(np.size(df_test, 0))

df_test['price_doc'] = df_train['price_doc'].mean()

df = pd.concat([df_train, df_test])
print(np.size(df,0))

30471
7662
38133


## Understanding Data
1. Prepare Initial Report.(Dtype, sample_data, max, min, mean, std, median,Correlation with price_doc, top_corr, missing_values_count, missing_values_percentage, Drop it, Importance_level, Comment)
2. Go though full data (all columns) and figure out 10% of most important feature (Using corelation and logic).
3. Droping unnecessary tables.
4. Molding data that is not showing proper corelation. (create, modify ...etc)


In [3]:
def initial_report(df, output_class, path):
    '''
    Returns a csv file that is very usefull in making initial assumptions

    Parameters:
    -----------
        df : Dataframe
        Output Class : Class to predict (string)
        path : Path for csv to Save

    Returns:
    --------
        Nothing
    
    '''
    df_row_count = np.size(df, axis=0)
    columns = df.columns
    df_na = df.dropna()
    report_csv = pd.DataFrame({}, index=columns.values)
    corr = df.corr()

    # Adding Dtypes Info
    report_csv['Dtype'] = df.dtypes

    # Adding 5 sample to read from
    for i in range(1, 6):
        report_csv.loc[:, 'sample_data_'+str(i)] = df_na.iloc[i]

    # other imp data
    report_csv['max'] = df_na.max()
    report_csv['min'] = df_na.min()
    report_csv['mean'] = df_na.mean()
    report_csv['std'] = df_na.std()
    report_csv['median'] = df_na.median()

    for j in df.select_dtypes(exclude=['object']).columns:
        top_corr = list(corr[j].sort_values(ascending=False).index)
        for i in range(1, 6):
            report_csv.loc[j, 'top_corr_'+str(i)] = top_corr[i]

    # missing value
    report_csv['missing_values_count'] = df.isnull().sum()
    report_csv['missing_values_percentage'] = (report_csv['missing_values_count']/df_row_count)
    
    # Adding correlation with output class
    report_csv['Correlation with '+output_class] = corr[output_class]

    report_csv['Importance_level'] = "Average"
    report_csv['Comment'] = "N/a"

    report_csv.to_csv(path)

    
# initial_report(df_train, 'price_doc', 'final/initial_report.csv')

def initial_report_output(path):
    '''
    Load the conclusion from the CSV file
    
    Importance_level: Drop -- drop_list
                      High -- imp_list
    
    Comment -> Print Conclusions

    Parameters:
    -----------
        path : Path for csv to Save

    Returns:
    --------
        [drop_list, imp_list]
    
    '''
    report_csv = pd.read_csv(path)
    drop_feature_list = report_csv[report_csv.Importance_level == "Drop"].iloc[:,0]
    imp_feature_list = report_csv[report_csv.Importance_level == "High"].iloc[:,0]
    drop_list = list(drop_feature_list)
    imp_list = list(imp_feature_list)
    comments = report_csv[report_csv.Comment != "N/a"].iloc[:,[0,-1]]
    print('Comments: \n', comments, '\n\n\n\n')
    
    missing_table = report_csv[report_csv.Importance_level != "Drop"][report_csv.missing_values_percentage > 0].iloc[:,[0,1,18]]
    print('Missing Value: \n', missing_table, '\n\n\n\n')
    
    return [drop_list, imp_list]

[drop_list, imp_list] = initial_report_output('final/initial_report.csv')

Comments: 
      Unnamed: 0                                            Comment
85   build_year        Need to modify using year sold - year build
98     full_all  need to modify it to population density with area
126    material                       bin to classify proper group
170       floor     make a feature of roof house or 1 floor hosuse
181       state                       bin to classify proper group
289    sub_area                                bin and pop density
291   timestamp                       break it into Year and month 




Missing Value: 
                     Unnamed: 0    Dtype  missing_values_percentage
4                 school_quota  float64                   0.219389
5            build_count_panel  float64                   0.163795
8             build_count_slag  float64                   0.163795
11           build_count_frame  float64                   0.163795
12             build_count_mix  float64                   0.163795
13            build_count_wo

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:82: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


## Creating new features:
1. Breaking feature into possible sessions(date to month and week) or simplifications of existing feature or Bining values together .
2. Combination of existing feature (+, -, *, /,) (May use weights)(Completely manual) 
3. Remove Duplicate and unrelated features using (col vs y plot)
4. Polynomials of top feature (2nd, 3rd, 2nd root, 3rd root, exp) (Use algo to check corelations)

*Example Strategy for breaking data*
1. From name we can extract the position and status ex Dr. Ms. etc

In [4]:
# timestamp breaking
df['sold_year'], df['month'], _ = df['timestamp'].str.split('-').str

In [5]:
# Age of city
df['city_build'] = df[['build_count_1971-1995',
'build_count_1921-1945',
'build_count_after_1995',
'build_count_1946-1970',
'build_count_before_1920']].idxmax(axis=1).str.replace('er_','-').str.replace('re_','-').str.split('-').str[1]

In [6]:
# Material of city
df['city_build_material'] = df[['build_count_panel',
'build_count_slag',
'build_count_frame',
'build_count_mix',
'build_count_wood',
'build_count_foam',
'build_count_block',
'build_count_monolith',
'build_count_brick']].idxmax(axis=1).str.replace('build_count_','')

In [7]:
# city population distribution
df['pop_density'] = df.work_all/df.area_m

In [32]:
def corr(X,Y):
    df1 = pd.DataFrame()
    df1['x'] = X
    df1['y'] = Y
    return df1.corr().iloc[1,0]

def corr_poly(df, output_class, varible_name):
    output = df[output_class]
    for i, col in enumerate(df.select_dtypes(exclude=['object']).columns):
        if df[col].min() > 0:
            column_data = [0,0,0,0,0,0,0,0]
            column_data[0] = df[col]
            column_data[1] = column_data[0]**2
            column_data[2] = column_data[0]**3
            column_data[3] = column_data[0]**0.5
            column_data[4] = column_data[0]**(1/3)
            column_data[5] = np.expm1(column_data[0])
            column_data[6] = np.log1p(column_data[0])
            column_data[7] = 1/(1+column_data[0])

            corr_column_data = [0,0,0,0,0,0,0,0]
            for j in range(7):
                corr_column_data[j] = abs(corr(column_data[j],output))

            k = corr_column_data.index(max(corr_column_data))

            if k == 1:
                print(varible_name+'["'+col+'"] = '+varible_name+'["'+col+'"]**2')
            elif k == 2:
                print(varible_name+'["'+col+'"] = '+varible_name+'["'+col+'"]**3')
            elif k == 3:
                print(varible_name+'["'+col+'"] = '+varible_name+'["'+col+'"]**0.5')
            elif k == 4:
                print(varible_name+'["'+col+'"] = '+varible_name+'["'+col+'"]**(1/3)')
            elif k == 5:
                print(varible_name+'["'+col+'"] = np.expm1('+varible_name+'["'+col+'"])')
            elif k == 6:
                print(varible_name+'["'+col+'"] = np.log1p('+varible_name+'["'+col+'"])')
            elif k == 7:
                print(varible_name+'["'+col+'"] = 1/(1+'+varible_name+'["'+col+'"])')
        
            
# corr_poly(df, 'price_doc', 'df')      

df["material"] = df["material"]**3
df["state"] = df["state"]**3
df["area_m"] = df["area_m"]**3
df["school_quota"] = np.log1p(df["school_quota"])
df["water_treatment_km"] = np.log1p(df["water_treatment_km"])
df["incineration_km"] = np.expm1(df["incineration_km"])
df["railroad_station_avto_km"] = df["railroad_station_avto_km"]**3
df["public_transport_station_min_walk"] = df["public_transport_station_min_walk"]**2
df["mkad_km"] = np.expm1(df["mkad_km"])
df["big_road1_km"] = df["big_road1_km"]**2
df["big_road2_km"] = df["big_road2_km"]**3
df["railroad_km"] = df["railroad_km"]**3
df["bus_terminal_avto_km"] = df["bus_terminal_avto_km"]**3
df["oil_chemistry_km"] = np.expm1(df["oil_chemistry_km"])
df["nuclear_reactor_km"] = df["nuclear_reactor_km"]**3
df["power_transmission_line_km"] = df["power_transmission_line_km"]**2
df["market_shop_km"] = df["market_shop_km"]**3
df["hospice_morgue_km"] = df["hospice_morgue_km"]**2
df["detention_facility_km"] = np.expm1(df["detention_facility_km"])
df["uni

In [33]:
df["material"] = df["material"]**3
df["state"] = df["state"]**3
df["area_m"] = df["area_m"]**3
df["school_quota"] = np.log1p(df["school_quota"])
df["water_treatment_km"] = np.log1p(df["water_treatment_km"])
df["incineration_km"] = np.expm1(df["incineration_km"])
df["railroad_station_avto_km"] = df["railroad_station_avto_km"]**3
df["public_transport_station_min_walk"] = df["public_transport_station_min_walk"]**2
df["mkad_km"] = np.expm1(df["mkad_km"])
df["big_road1_km"] = df["big_road1_km"]**2
df["big_road2_km"] = df["big_road2_km"]**3
df["railroad_km"] = df["railroad_km"]**3
df["bus_terminal_avto_km"] = df["bus_terminal_avto_km"]**3
df["oil_chemistry_km"] = np.expm1(df["oil_chemistry_km"])
df["nuclear_reactor_km"] = df["nuclear_reactor_km"]**3
df["power_transmission_line_km"] = df["power_transmission_line_km"]**2
df["market_shop_km"] = df["market_shop_km"]**3
df["hospice_morgue_km"] = df["hospice_morgue_km"]**2
df["detention_facility_km"] = np.expm1(df["detention_facility_km"])
df["university_km"] = df["university_km"]**2
df["museum_km"] = df["museum_km"]**3
df["exhibition_km"] = df["exhibition_km"]**2
df["catering_km"] = np.log1p(df["catering_km"])
df["green_part_5000"] = df["green_part_5000"]**3
df["pop_density"] = df["pop_density"]**(1/3)

# Droping Unnecessary Columns

In [9]:
# Drop UnNecessary Columns
drop_column = ['0_13_female','0_13_male','0_17_all','0_17_female','0_17_male','0_6_all','0_6_female','0_6_male','16_29_all','16_29_female','16_29_male','7_14_all','7_14_female','7_14_male','big_church_count_1000','big_church_count_1500','big_church_count_2000','big_church_count_3000','big_church_count_500','big_church_count_5000','big_market_km','build_count_1921-1945','build_count_1946-1970','build_count_1971-1995','build_count_after_1995','build_count_before_1920','cafe_avg_price_1000','cafe_avg_price_1500','cafe_avg_price_2000','cafe_avg_price_3000','cafe_avg_price_500','cafe_avg_price_5000','cafe_count_1000_na_price','cafe_count_1000_price_1000','cafe_count_1000_price_1500','cafe_count_1000_price_2500','cafe_count_1000_price_4000','cafe_count_1000_price_500','cafe_count_1000_price_high','cafe_count_1500','cafe_count_1500_na_price','cafe_count_1500_price_1000','cafe_count_1500_price_1500','cafe_count_1500_price_2500','cafe_count_1500_price_4000','cafe_count_1500_price_500','cafe_count_1500_price_high','cafe_count_2000_na_price','cafe_count_2000_price_1000','cafe_count_2000_price_1500','cafe_count_2000_price_2500','cafe_count_2000_price_4000','cafe_count_2000_price_500','cafe_count_2000_price_high','cafe_count_3000_na_price','cafe_count_3000_price_1000','cafe_count_3000_price_1500','cafe_count_3000_price_2500','cafe_count_3000_price_4000','cafe_count_3000_price_500','cafe_count_3000_price_high','cafe_count_500_na_price','cafe_count_500_price_1000','cafe_count_500_price_1500','cafe_count_500_price_2500','cafe_count_500_price_4000','cafe_count_500_price_500','cafe_count_500_price_high','cafe_count_5000_na_price','cafe_count_5000_price_1000','cafe_count_5000_price_1500','cafe_count_5000_price_2500','cafe_count_5000_price_4000','cafe_count_5000_price_500','cafe_count_5000_price_high','cafe_sum_1000_max_price_avg','cafe_sum_1000_min_price_avg','cafe_sum_1500_max_price_avg','cafe_sum_1500_min_price_avg','cafe_sum_2000_max_price_avg','cafe_sum_2000_min_price_avg','cafe_sum_3000_max_price_avg','cafe_sum_3000_min_price_avg','cafe_sum_500_max_price_avg','cafe_sum_500_min_price_avg','cafe_sum_5000_max_price_avg','cafe_sum_5000_min_price_avg','church_count_1000','church_count_1500','church_count_2000','church_count_3000','church_count_500','ekder_female','ekder_male','female_f','green_part_1000','green_part_1500','green_part_2000','green_part_3000','green_part_500','green_zone_km','green_zone_part','id','ID_big_road1','ID_big_road2','ID_bus_terminal','ID_metro','ID_railroad_station_avto','ID_railroad_station_walk','ID_railroad_terminal','leisure_count_1000','leisure_count_1500','leisure_count_2000','leisure_count_3000','leisure_count_500','male_f','market_count_1000','market_count_1500','market_count_2000','market_count_3000','market_count_500','mosque_count_1000','mosque_count_1500','mosque_count_2000','mosque_count_3000','mosque_count_500','office_count_1000','office_count_1500','office_count_2000','office_count_3000','office_count_500','office_count_5000','office_raion','office_sqm_1000','office_sqm_1500','office_sqm_2000','office_sqm_3000','office_sqm_500','prom_part_1000','prom_part_1500','prom_part_2000','prom_part_3000','prom_part_500','prom_part_5000','raion_build_count_with_builddate_info','raion_build_count_with_material_info','sport_count_1000','sport_count_1500','sport_count_2000','sport_count_3000','sport_objects_raion','trc_count_1000','trc_count_1500','trc_count_2000','trc_count_500','trc_count_5000','trc_sqm_1000','trc_sqm_1500','trc_sqm_2000','trc_sqm_3000','trc_sqm_500','trc_sqm_5000','work_female','work_male','young_female','young_male']

def find_similar_feature(df, output_class_name, threshold=0.75):
    df = pd.DataFrame(df)
    corr_with_out = df[output_class_name]

    df.drop(output_class_name, 1, inplace=True)
    df.drop(output_class_name, 0, inplace=True)

    row_count = np.size(df, axis=0)
    col_count = np.size(df, axis=1)
    
    if row_count != col_count:
        raise ValueError('Data is Unsymmetric')
        
    drop_list = pd.DataFrame()
    
    for i in range(0, row_count):
        for j in range(0, i):
            if df.iloc[i,j] >= threshold:
                if(abs(corr_with_out[i]) >= abs(corr_with_out[j])):
                    drop_list = drop_list.append([df.columns[i]])
                else:
                    drop_list = drop_list.append([df.columns[j]])
#                 print(df.columns[i], df.index[j],"are simmilar", df.iloc[i,j], "with ", corr_with_out[i])
                
    return drop_list.drop_duplicates()[0]

corr = df.corr()
drop_list = find_similar_feature(corr, "price_doc", threshold=0.90)
drop_column.extend(drop_list)
df.drop(drop_column, 1, inplace=True)

## Some important data-modification

In [10]:
def fill_missing_data_with_suitable(df):
    # missing value fill with mode in case of Categorical feature
    for i in df.select_dtypes(include=['object']).columns:
        try:
            df[i] = df[i].fillna(df[i].value_counts()[0])
        except:
            print(i)
    
    # missing value fill with mean in case of Numerical column
    for i in df.select_dtypes(exclude=['object']).columns:
        df[i] = df[i].fillna(df[i].mean())
    
    return df

df.replace([np.inf, -np.inf], np.nan)
df = fill_missing_data_with_suitable(df)

In [11]:
df = pd.get_dummies(df)

In [12]:
# Normalization
from scipy.stats import skew

skewed = df.select_dtypes(exclude=['object']).apply(lambda x: skew(x.astype(float)))
skewed = skewed[abs(skewed) > 0.75].index
df[skewed] = np.log1p(df[skewed])

In [34]:
df_train = df.iloc[0:30471, :]
df_test = df.iloc[30471:, :]

In [35]:
outlier_id = [3527, 13546, 23584, 20722, 21734, 21852, 25940, 10089, 11621,
       26713, 10368, 13117, 21415, 28734, 11701, 12742, 14426, 14685,
       14729, 14895, 15868, 15902, 16790, 17010, 17059, 17396, 17404,
       17761, 18074, 18356, 18959, 20324, 20863, 21565, 21663, 21938,
       22487, 23397, 23483, 23656, 24820, 25769, 25826, 25876, 26923,
       27371, 28074, 29136,  7977, 22694, 16308, 22778,   723,   948,
        1402,  2934,  6191,  8895, 10901, 11278, 12282, 17296, 18933,
       18990, 22664, 23602, 26487,  8073,  9502, 29656,  8933,  1465,
       10587, 16486, 28326,  7457,  2118]

df_train.drop(outlier_id, 0, inplace=True)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [36]:
X = np.array(df_train.drop(["price_doc"], 1))
X = preprocessing.scale(X)
y = np.array(df_train["price_doc"])

X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.5)

print(np.size(df_train, 0))
print(np.size(df_test, 0))

30394
7662


In [37]:
from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [41]:
import xgboost as xgb

regr = xgb.XGBRegressor(booster="gbtree", learning_rate=0.1)
#                  colsample_bytree=0.2,
#                  gamma=0.0,
#                  learning_rate=0.05,
#                  max_depth=6,
#                  min_child_weight=1.5,
#                  n_estimators=7200,                                                                  
#                  reg_alpha=0.9,
#                  reg_lambda=0.6,
#                  subsample=0.2,
#                  seed=42,
#                  silent=1

regr.fit(X_train, y_train)

y_train_ = regr.predict(X_train)
print("Training Error: ",rmse(y_train,y_train_))

y_test_ = regr.predict(X_test)
print("Testing Error: ",rmse(y_test,y_test_))

Training Error:  0.443215597196
Testing Error:  0.472703522299


In [39]:
X_sub = np.array(df_test.drop(["price_doc"], 1))
X_sub = preprocessing.scale(X_sub)
y_sub = regr.predict(X_sub)

In [40]:
output['price_doc'] = np.expm1(y_sub)
output.to_csv("submission_data.csv", index=False)